### 직방
- 동 이름을 입력하면 아파트 매물을 데이터 프레임으로 만들기
- robots.txt

#### 1. 웹서비스 분석 : URL 찾기, 크롤링 전략 세우기
- 동 이름 입력 -> lat, lng
- lat, lng -> geohash : geohash 패키지 설치
- geohash -> item ids
- item ids -> item data (데이터 프레임)

- From zigbang.py

In [60]:
import geohash2

# 함수로 만들기    
def oneroom(addr):
    
    # 1. 동이름으로 위도 경도 구하기
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    lat, lng = response.json()["items"][0]["lat"], response.json()["items"][0]["lng"]
    
    # 2. 위도 경도로 geohash 알아내기
    geohash = geohash2.encode(lat, lng, precision=5) 
    
    # 3. geohash로 매물 리스트 가져오기
    url = "https://apis.zigbang.com/v2/items?\
deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    # 4. id로 매물 정보 가져오기
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "false",
        "item_ids": ids[:900],
    }

    response = requests.post(url, params)
    items = response.json()["items"]
    return [item for item in items if addr in item["address"]]


In [61]:
data = oneroom("명일동")
item_df = pd.DataFrame(data)
item_df.tail(3)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
29,None,23756914,https://ic.zigbang.com/ic/items/23756914/1.jpg,월세,월세,1000,57,38.0,"{'m2': 38, 'p': '11.5'}","{'m2': 19.83, 'p': '6'}",None,None,고,고,15,"넓은B타입☆고덕역 도보1분,신축1년차오피스텔",None,01,강동구 명일동,"{'lat': 37.55423857729919, 'lng': 127.15435048...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,9,2020-09-02T17:06:56+09:00,True
30,None,23771854,https://ic.zigbang.com/ic/items/23771854/1.jpg,전세,전세,14000,0,34.0,"{'m2': 34, 'p': '10.3'}","{'m2': 17.86, 'p': '5.4'}",None,None,고,고,15,신축1년-고덕역 도보1분☆전세자금대출ok☆,None,01,강동구 명일동,"{'lat': 37.554231799236526, 'lng': 127.1549280...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,8,2020-09-03T14:52:28+09:00,True
31,None,23767890,https://ic.zigbang.com/ic/items/23767890/1.jpg,월세,월세,1000,65,45.0,"{'m2': 45, 'p': '13.6'}","{'m2': 19.5, 'p': '5.9'}",None,None,저,저,20,고덕역 바로 앞. 신축 첫 입주 효성해링턴오피스텔,None,01,강동구 명일동,"{'lat': 37.55449419506087, 'lng': 127.15495838...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,9,2020-09-03T12:22:58+09:00,True


In [62]:
len(item_df.columns)

30

In [63]:
# pandas option 설정
pd.options.display.max_columns = len(item_df.columns)

In [64]:
item_df.tail(3)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
29,None,23756914,https://ic.zigbang.com/ic/items/23756914/1.jpg,월세,월세,1000,57,38.0,"{'m2': 38, 'p': '11.5'}","{'m2': 19.83, 'p': '6'}",None,None,고,고,15,"넓은B타입☆고덕역 도보1분,신축1년차오피스텔",None,01,강동구 명일동,"{'lat': 37.55423857729919, 'lng': 127.15435048...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,9,2020-09-02T17:06:56+09:00,True
30,None,23771854,https://ic.zigbang.com/ic/items/23771854/1.jpg,전세,전세,14000,0,34.0,"{'m2': 34, 'p': '10.3'}","{'m2': 17.86, 'p': '5.4'}",None,None,고,고,15,신축1년-고덕역 도보1분☆전세자금대출ok☆,None,01,강동구 명일동,"{'lat': 37.554231799236526, 'lng': 127.1549280...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,8,2020-09-03T14:52:28+09:00,True
31,None,23767890,https://ic.zigbang.com/ic/items/23767890/1.jpg,월세,월세,1000,65,45.0,"{'m2': 45, 'p': '13.6'}","{'m2': 19.5, 'p': '5.9'}",None,None,저,저,20,고덕역 바로 앞. 신축 첫 입주 효성해링턴오피스텔,None,01,강동구 명일동,"{'lat': 37.55449419506087, 'lng': 127.15495838...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,9,2020-09-03T12:22:58+09:00,True


In [65]:
columns = ['item_id', 'building_floor', 'floor_string', 'size_m2', '공급면적', 'title', 'rent', 'random_location']
item_df = item_df[columns]
item_df.tail(3)

,item_id,building_floor,floor_string,size_m2,공급면적,title,rent,random_location
29,23756914,15,고,38.0,"{'m2': 38, 'p': '11.5'}","넓은B타입☆고덕역 도보1분,신축1년차오피스텔",57,"{'lat': 37.55423857729919, 'lng': 127.15435048..."
30,23771854,15,고,34.0,"{'m2': 34, 'p': '10.3'}",신축1년-고덕역 도보1분☆전세자금대출ok☆,0,"{'lat': 37.554231799236526, 'lng': 127.1549280..."
31,23767890,20,저,45.0,"{'m2': 45, 'p': '13.6'}",고덕역 바로 앞. 신축 첫 입주 효성해링턴오피스텔,65,"{'lat': 37.55449419506087, 'lng': 127.15495838..."


In [66]:
item_df['m2'] = item_df['공급면적'].apply(lambda data: data['m2'])
item_df['p'] = item_df['공급면적'].apply(lambda data: data['p'])
item_df.drop(columns=['공급면적'], inplace=True)
item_df.tail()

,item_id,building_floor,floor_string,size_m2,title,rent,random_location,m2,p
27,23297225,15,중,39.66,고덕역갑을명가시티 ☆분리형원룸☆,80,"{'lat': 37.553740677655725, 'lng': 127.1543329...",39.66,12
28,23679620,15,고,34.72,☆풀옵션 단기임대☆방역완료,85,"{'lat': 37.55375892374511, 'lng': 127.15430852...",34.72,10.5
29,23756914,15,고,38.00,"넓은B타입☆고덕역 도보1분,신축1년차오피스텔",57,"{'lat': 37.55423857729919, 'lng': 127.15435048...",38.00,11.5
30,23771854,15,고,34.00,신축1년-고덕역 도보1분☆전세자금대출ok☆,0,"{'lat': 37.554231799236526, 'lng': 127.1549280...",34.00,10.3
31,23767890,20,저,45.00,고덕역 바로 앞. 신축 첫 입주 효성해링턴오피스텔,65,"{'lat': 37.55449419506087, 'lng': 127.15495838...",45.00,13.6


In [67]:
item_df.drop(columns=['size_m2'], inplace=True)

In [68]:
item_df.tail()

,item_id,building_floor,floor_string,title,rent,random_location,m2,p
27,23297225,15,중,고덕역갑을명가시티 ☆분리형원룸☆,80,"{'lat': 37.553740677655725, 'lng': 127.1543329...",39.66,12
28,23679620,15,고,☆풀옵션 단기임대☆방역완료,85,"{'lat': 37.55375892374511, 'lng': 127.15430852...",34.72,10.5
29,23756914,15,고,"넓은B타입☆고덕역 도보1분,신축1년차오피스텔",57,"{'lat': 37.55423857729919, 'lng': 127.15435048...",38.00,11.5
30,23771854,15,고,신축1년-고덕역 도보1분☆전세자금대출ok☆,0,"{'lat': 37.554231799236526, 'lng': 127.1549280...",34.00,10.3
31,23767890,20,저,고덕역 바로 앞. 신축 첫 입주 효성해링턴오피스텔,65,"{'lat': 37.55449419506087, 'lng': 127.15495838...",45.00,13.6


In [75]:
# 명일동에서 15층 이상되는 고층 원룸에 월세 찾아서 구매?
items_df = pd.DataFrame(oneroom("명일동"))
items_df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
30,None,23771854,https://ic.zigbang.com/ic/items/23771854/1.jpg,전세,전세,14000,0,34.0,"{'m2': 34, 'p': '10.3'}","{'m2': 17.86, 'p': '5.4'}",None,None,고,고,15,신축1년-고덕역 도보1분☆전세자금대출ok☆,None,01,강동구 명일동,"{'lat': 37.5542148478738, 'lng': 127.154313338...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,8,2020-09-03T14:52:28+09:00,True
31,None,23767890,https://ic.zigbang.com/ic/items/23767890/1.jpg,월세,월세,1000,65,45.0,"{'m2': 45, 'p': '13.6'}","{'m2': 19.5, 'p': '5.9'}",None,None,저,저,20,고덕역 바로 앞. 신축 첫 입주 효성해링턴오피스텔,None,01,강동구 명일동,"{'lat': 37.55495597631825, 'lng': 127.15497308...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,9,2020-09-03T12:22:58+09:00,True


In [76]:
# items_df.dtypes    (object)
# items_df["building_floor"].astype("int")
items_df = items_df[items_df["building_floor"].notnull()]
items_df.reset_index(drop=True, inplace=True)
items_df

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,23660038,https://ic.zigbang.com/ic/items/23660038/1.jpg,월세,월세,10000,15,23.14,"{'m2': 23.14, 'p': '7'}","{'m2': 23.14, 'p': '7'}",None,None,3,3,5,굽은다리역 도보 3분 초역세권 보증금대출 가능,None,01,강동구 명일동,"{'lat': 37.54513649256757, 'lng': 127.14396730...",False,True,원룸,[추천],서울시 강동구 명일동,None,None,3.5,2020-08-27T15:33:03+09:00,False
1,None,23403356,https://ic.zigbang.com/ic/items/23403356/1.jpg,전세,전세,16000,0,56.20,"{'m2': 56.2, 'p': '17'}","{'m2': 42.98, 'p': '13'}",None,None,3,3,4,💎✨굽은다리 초 역세권💎✨투룸 전세자금 대출 환영,None,04,강동구 명일동,"{'lat': 37.545204704097365, 'lng': 127.1446887...",False,True,빌라,[],서울시 강동구 명일동,None,None,2,2020-08-12T13:34:24+09:00,False
2,None,23535541,https://ic.zigbang.com/ic/items/23535541/1.jpg,전세,전세,27000,0,72.72,"{'m2': 72.72, 'p': '22'}","{'m2': 56.19, 'p': '17'}",None,None,6,6,6,💟층간소음NO💢신축2년💢병렬주차💢에어컨2대💢,None,05,강동구 명일동,"{'lat': 37.54494388366286, 'lng': 127.14404971...",False,True,빌라,[],서울시 강동구 명일동,None,None,5.5,2020-08-20T15:01:25+09:00,False
3,None,22858368,https://ic.zigbang.com/ic/items/22858368/1.jpg,월세,월세,8000,65,47.83,"{'m2': 47.83, 'p': '14.5'}","{'m2': 26.45, 'p': '8'}",None,None,고,고,15,!!!서울시 청년 보증금 대출로 입주가능한집!!!,None,03,강동구 명일동,"{'lat': 37.54516095251998, 'lng': 127.14362712...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,10,2020-08-25T16:35:12+09:00,False
4,None,23204774,https://ic.zigbang.com/ic/items/23204774/1.jpg,월세,월세,2000,80,39.67,"{'m2': 39.67, 'p': '12'}","{'m2': 29.75, 'p': '9'}",None,None,중,중,15,굽은다리역30초 신축 분리형 풀옵션,None,02,강동구 명일동,"{'lat': 37.545152193163496, 'lng': 127.1435891...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,8,2020-08-21T14:32:03+09:00,False
5,None,23312137,https://ic.zigbang.com/ic/items/23312137/1.jpg,월세,월세,1000,70,46.28,"{'m2': 46.28, 'p': '14'}","{'m2': 23.14, 'p': '7'}",None,None,고,고,15,5호선 굽은다리역 초역세권 신축 첫입주,None,03,강동구 명일동,"{'lat': 37.545654448690186, 'lng': 127.1430003...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,6,2020-08-24T14:57:09+09:00,False
6,None,23356748,https://ic.zigbang.com/ic/items/23356748/1.jpg,월세,월세,3000,70,49.59,"{'m2': 49.59, 'p': '15'}","{'m2': 33.16, 'p': '10'}",None,None,고,고,15,청년임차보증금대출 브릿지/5호선굽은다리4번출구에서 한발자국,None,03,강동구 명일동,"{'lat': 37.5451568209894, 'lng': 127.143002217...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,7,2020-08-21T11:04:39+09:00,False
7,None,23431153,https://ic.zigbang.com/ic/items/23431153/1.jpg,월세,월세,1000,70,33.06,"{'m2': 33.06, 'p': '10'}","{'m2': 29.75, 'p': '9'}",None,None,고,고,15,★복층 역세권 풀옵션 신축,None,03,강동구 명일동,"{'lat': 37.545171997181775, 'lng': 127.1430173...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,8,2020-09-01T10:44:36+09:00,False
8,None,23437790,https://ic.zigbang.com/ic/items/23437790/1.jpg,월세,월세,2000,65,52.89,"{'m2': 52.89, 'p': '16'}","{'m2': 26.45, 'p': '8'}",None,None,고,고,15,"💎굽은다리역4걸음,휘트니스텐터이용가능/최첨단건물",None,03,강동구 명일동,"{'lat': 37.5451474663661, 'lng': 127.143019339...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,7,2020-08-21T11:03:16+09:00,False
9,None,23477435,https://ic.zigbang.com/ic/items/23477435/1.jpg,월세,월세,1000,70,31.74,"{'m2': 31.74, 'p': '9.6'}","{'m2': 19.83, 'p': '6'}",None,None,중,중,15,##굽은다리역 4번 출구바로앞 스세권 복층형원룸 ##,None,03,강동구 명일동,"{'lat': 37.54566243009366, 'lng': 127.14300398...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,7,2020-08-18T11:11:49+09:00,False


In [77]:
items_df.dtypes

section_type         object
item_id               int64
images_thumbnail     object
sales_type           object
sales_title          object
deposit               int64
rent                  int64
size_m2             float64
공급면적                 object
전용면적                 object
계약면적                 object
room_type_title      object
floor                object
floor_string         object
building_floor       object
title                object
is_first_movein      object
room_type            object
address              object
random_location      object
is_zzim                bool
status                 bool
service_type         object
tags                 object
address1             object
address2             object
address3             object
manage_cost          object
reg_date             object
is_new                 bool
dtype: object

In [78]:
items_df["building_floor"] = items_df["building_floor"].astype(int)

In [79]:
items_df.dtypes

section_type         object
item_id               int64
images_thumbnail     object
sales_type           object
sales_title          object
deposit               int64
rent                  int64
size_m2             float64
공급면적                 object
전용면적                 object
계약면적                 object
room_type_title      object
floor                object
floor_string         object
building_floor        int32
title                object
is_first_movein      object
room_type            object
address              object
random_location      object
is_zzim                bool
status                 bool
service_type         object
tags                 object
address1             object
address2             object
address3             object
manage_cost          object
reg_date             object
is_new                 bool
dtype: object

In [85]:
items_df[items_df["building_floor"] >= 15]

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
3,None,22858368,https://ic.zigbang.com/ic/items/22858368/1.jpg,월세,월세,8000,65,47.83,"{'m2': 47.83, 'p': '14.5'}","{'m2': 26.45, 'p': '8'}",None,None,고,고,15,!!!서울시 청년 보증금 대출로 입주가능한집!!!,None,03,강동구 명일동,"{'lat': 37.54516095251998, 'lng': 127.14362712...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,10,2020-08-25T16:35:12+09:00,False
4,None,23204774,https://ic.zigbang.com/ic/items/23204774/1.jpg,월세,월세,2000,80,39.67,"{'m2': 39.67, 'p': '12'}","{'m2': 29.75, 'p': '9'}",None,None,중,중,15,굽은다리역30초 신축 분리형 풀옵션,None,02,강동구 명일동,"{'lat': 37.545152193163496, 'lng': 127.1435891...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,8,2020-08-21T14:32:03+09:00,False
5,None,23312137,https://ic.zigbang.com/ic/items/23312137/1.jpg,월세,월세,1000,70,46.28,"{'m2': 46.28, 'p': '14'}","{'m2': 23.14, 'p': '7'}",None,None,고,고,15,5호선 굽은다리역 초역세권 신축 첫입주,None,03,강동구 명일동,"{'lat': 37.545654448690186, 'lng': 127.1430003...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,6,2020-08-24T14:57:09+09:00,False
6,None,23356748,https://ic.zigbang.com/ic/items/23356748/1.jpg,월세,월세,3000,70,49.59,"{'m2': 49.59, 'p': '15'}","{'m2': 33.16, 'p': '10'}",None,None,고,고,15,청년임차보증금대출 브릿지/5호선굽은다리4번출구에서 한발자국,None,03,강동구 명일동,"{'lat': 37.5451568209894, 'lng': 127.143002217...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,7,2020-08-21T11:04:39+09:00,False
7,None,23431153,https://ic.zigbang.com/ic/items/23431153/1.jpg,월세,월세,1000,70,33.06,"{'m2': 33.06, 'p': '10'}","{'m2': 29.75, 'p': '9'}",None,None,고,고,15,★복층 역세권 풀옵션 신축,None,03,강동구 명일동,"{'lat': 37.545171997181775, 'lng': 127.1430173...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,8,2020-09-01T10:44:36+09:00,False
8,None,23437790,https://ic.zigbang.com/ic/items/23437790/1.jpg,월세,월세,2000,65,52.89,"{'m2': 52.89, 'p': '16'}","{'m2': 26.45, 'p': '8'}",None,None,고,고,15,"💎굽은다리역4걸음,휘트니스텐터이용가능/최첨단건물",None,03,강동구 명일동,"{'lat': 37.5451474663661, 'lng': 127.143019339...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,7,2020-08-21T11:03:16+09:00,False
9,None,23477435,https://ic.zigbang.com/ic/items/23477435/1.jpg,월세,월세,1000,70,31.74,"{'m2': 31.74, 'p': '9.6'}","{'m2': 19.83, 'p': '6'}",None,None,중,중,15,##굽은다리역 4번 출구바로앞 스세권 복층형원룸 ##,None,03,강동구 명일동,"{'lat': 37.54566243009366, 'lng': 127.14300398...",False,True,오피스텔,[추천],서울시 강동구 명일동,None,None,7,2020-08-18T11:11:49+09:00,False
10,None,23485263,https://ic.zigbang.com/ic/items/23485263/1.jpg,월세,월세,2000,80,36.36,"{'m2': 36.36, 'p': '11'}","{'m2': 23.14, 'p': '7'}",None,None,중,중,15,!!!스세권 신축 첫입주 복층형 원룸 !!!,None,03,강동구 명일동,"{'lat': 37.54566225350248, 'lng': 127.14299600...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,8,2020-08-25T16:35:10+09:00,False
11,None,23505137,https://ic.zigbang.com/ic/items/23505137/1.jpg,월세,월세,3000,60,29.75,"{'m2': 29.75, 'p': '9'}","{'m2': 23.14, 'p': '7'}",None,None,고,고,15,💟청년임차보증금대출가능한신축복층오피💟1층보안및홈오토시설,None,03,강동구 명일동,"{'lat': 37.54514170095169, 'lng': 127.14363904...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,7,2020-09-03T13:09:41+09:00,True
12,None,23564250,https://ic.zigbang.com/ic/items/23564250/1.jpg,월세,월세,8000,70,69.42,"{'m2': 69.42, 'p': '21'}","{'m2': 35.7, 'p': '10.8'}",None,None,고,고,15,💎서울시청년융자추천브릿지/분리형/휘트니스구비함.,None,02,강동구 명일동,"{'lat': 37.54514041723252, 'lng': 127.14301521...",False,True,오피스텔,[],서울시 강동구 명일동,None,None,7,2020-08-21T16:07:52+09:00,False


In [84]:
result_df = items_df[
    (items_df["building_floor"] >= 15) &
    (items_df["floor_string"] == "고") &
    (items_df["sales_type"] == "월세")
]
len(result_df)

14

In [89]:
result_df["deposit"]

3      8000
5      1000
6      3000
7      1000
8      2000
11     3000
12     8000
13     8000
15     1000
16    10000
18     1000
19     6000
28       85
29     1000
Name: deposit, dtype: int64

In [90]:
np.average(result_df["deposit"])

3791.785714285714

In [88]:
np.min(result_df["deposit"])

85

In [6]:
import geohash2

# 함수로 만들기    
def crawl_oneroom(addr):
    
    # 1. 동이름으로 위도 경도 구하기
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    lat, lng = response.json()["items"][0]["lat"], response.json()["items"][0]["lng"]
    
    # 2. 위도 경도로 geohash 알아내기
    geohash = geohash2.encode(lat, lng, precision=5) 
    
    # 3. geohash로 매물 리스트 가져오기
    url = "https://apis.zigbang.com/v2/items?\
deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    # 4. id로 매물 정보 가져오기
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "false",
        "item_ids": ids[:900],
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    data = [item for item in items if addr in item["address"]]
    item_df = pd.DataFrame(data)
    pd.options.display.max_columns = len(item_df.columns)
    columns = ['item_id', 'building_floor', 'floor_string', 'size_m2', '공급면적', 'title', 'rent', 'random_location']
    item_df = item_df[columns]
    item_df['m2'] = item_df['공급면적'].apply(lambda data: data['m2'])
    item_df['p'] = item_df['공급면적'].apply(lambda data: data['p'])
    item_df.drop(columns=['공급면적'], inplace=True)
    item_df.drop(columns=['size_m2'], inplace=True)
    return item_df

In [7]:
crawl_oneroom("명일동")

,item_id,building_floor,floor_string,title,rent,random_location,m2,p
0,23660038,5,3,굽은다리역 도보 3분 초역세권 보증금대출 가능,15,"{'lat': 37.54462089648076, 'lng': 127.14397032...",23.14,7
1,23403356,4,3,💎✨굽은다리 초 역세권💎✨투룸 전세자금 대출 환영,0,"{'lat': 37.544713052681914, 'lng': 127.1440482...",56.20,17
2,23535541,6,6,💟층간소음NO💢신축2년💢병렬주차💢에어컨2대💢,0,"{'lat': 37.5449399207494, 'lng': 127.143451289...",72.72,22
3,22858368,15,고,!!!서울시 청년 보증금 대출로 입주가능한집!!!,65,"{'lat': 37.54562881776298, 'lng': 127.14361410...",47.83,14.5
4,23204774,15,중,굽은다리역30초 신축 분리형 풀옵션,80,"{'lat': 37.54564238165969, 'lng': 127.14361034...",39.67,12
5,23312137,15,고,5호선 굽은다리역 초역세권 신축 첫입주,70,"{'lat': 37.54515626160019, 'lng': 127.14297356...",46.28,14
6,23356748,15,고,청년임차보증금대출 브릿지/5호선굽은다리4번출구에서 한발자국,70,"{'lat': 37.54564150016988, 'lng': 127.14359308...",49.59,15
7,23431153,15,고,★복층 역세권 풀옵션 신축,70,"{'lat': 37.54515564761847, 'lng': 127.14363779...",33.06,10
8,23437790,15,고,"💎굽은다리역4걸음,휘트니스텐터이용가능/최첨단건물",65,"{'lat': 37.545140785209256, 'lng': 127.1436287...",52.89,16
9,23477435,15,중,##굽은다리역 4번 출구바로앞 스세권 복층형원룸 ##,70,"{'lat': 37.54515218683065, 'lng': 127.14300464...",31.74,9.6


In [8]:
%%writefile zigbang2.py


import pandas as pd
import requests
import geohash2


# 함수로 만들기    
def crawl_oneroom(addr):
    
    # 1. 동이름으로 위도 경도 구하기
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    lat, lng = response.json()["items"][0]["lat"], response.json()["items"][0]["lng"]
    
    # 2. 위도 경도로 geohash 알아내기
    geohash = geohash2.encode(lat, lng, precision=5) 
    
    # 3. geohash로 매물 리스트 가져오기
    url = "https://apis.zigbang.com/v2/items?\
deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    # 4. id로 매물 정보 가져오기
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "false",
        "item_ids": ids[:900],
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    data = [item for item in items if addr in item["address"]]
    item_df = pd.DataFrame(data)
    pd.options.display.max_columns = len(item_df.columns)
    columns = ['item_id', 'building_floor', 'floor_string', 'size_m2', '공급면적', 'title', 'rent', 'random_location']
    item_df = item_df[columns]
    item_df['m2'] = item_df['공급면적'].apply(lambda data: data['m2'])
    item_df['p'] = item_df['공급면적'].apply(lambda data: data['p'])
    item_df.drop(columns=['공급면적'], inplace=True)
    item_df.drop(columns=['size_m2'], inplace=True)
    return item_df

Overwriting zigbang2.py
